#  Importez toutes les libraries du notebook dans la prochaine cellule

In [114]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import seaborn as sns
from sklearn.linear_model import LinearRegression

# 1) Importez le jeu de données 'data.csv', puis affichez les cinq premières lignes

In [115]:
data = pd.read_table(filepath_or_buffer = "data.csv", # l'adresse du fichier
              header = 0, # noms de variables en ligne 1
              delimiter =',') # délimiteur, ici la virgule pour un csv
data.head()

,prenom,mois,annee,revenus_bruts,cotisation
0,Catherine,janvier,1979,1600,True
1,Catherine,février,1979,1600,True
2,Catherine,mars,1979,1600,True
3,Catherine,avril,1979,1600,True
4,Catherine,mai,1979,1600,True


# 2) Description des données


## 2) a-] La variable revenus_bruts est-elle qualitative ou quantitative?

### Answer :
the gross_income is a quantitative

## 2) b-] Quel est le type de la variable cotisation? Affichez le en console

### Answer:
The cotisation variable is a boolean variable

In [116]:
data.dtypes

prenom           object
mois             object
annee             int64
revenus_bruts     int64
cotisation         bool
dtype: object

## 2) c-] Affichez les dimensions du dataframe

### Answer: 

In [117]:
data.shape

(1968, 5)

## 2) d-] Combien de prénoms différents contient le dataset?

In [118]:
dy =data.set_index(["prenom","mois", "annee", "revenus_bruts", "cotisation"]).count(level="prenom")
dy


""
prenom
Catherine
Daniel
Marie
Patrick


the total number of prenom is 4

In [119]:
dy.shape

(4, 0)

# 3) Extraction des données et statistiques descriptives


## 3) a-] Quelle période couvre le dataset?

The data period is between 

In [120]:
#data['annee'].iloc[0], data['annee'].iloc[-1]
print("The data period is between " + str(data['annee'].iloc[0]) + " and " + str(data['annee'].iloc[-1]))

The data period is between 1979 and 2019


## 3) b-] Quelle est la moyenne des revenus bruts?

In [121]:
data['revenus_bruts'].mean()

2162.737804878049

## 3) c-] Quels ont été les revenus bruts de Patrick en janvier 1979?

In [122]:
pat = data.query("prenom == 'Patrick'")
mon = pat.query("mois == 'janvier'")
yr = mon.query("annee == '1979'")
value = yr.at[1476, 'revenus_bruts']
print(" Patrick's gross income in January 1979 is " + str(value))

 Patrick's gross income in January 1979 is 1700


## 3) d-] Quel montant brut a touché Catherine sur toute l'année 1983?

In [123]:
cat = data.query("prenom == 'Catherine'")
yrc = cat.query("annee == '1983'")
total = yrc['revenus_bruts'].sum()

print(" Catherine's gross income for the year 1983 is " + str(total))

 Catherine's gross income for the year 1983 is 19392


# 4) Transformation des données


## 4) a-] Créez un dataset contenant les revenus bruts annuels moyens de Patrick.

In [124]:
pat = data.query("prenom == 'Patrick'")
patAnnIncome = pat.groupby(["annee"]).mean()
del patAnnIncome['cotisation']
print(" Patrick's average annual gross income are: ")
patAnnIncome

 Patrick's average annual gross income are: 


,revenus_bruts
annee,
1979,1700
1980,1734
1981,1768
1982,1804
1983,1804
1984,1840
1985,1840
1986,1858
1987,1914


## 4) b-] Créez une variable revenus_nets à l'aide de la variable revenus_bruts.

Pour grossièrement passer d'un salaire brut à un salaire net, il faut retrancher 23% du salaire brut pour un employé du privé et 15% pour un employé de la fonction publique.

Patrick est employé de la fonction publique, les autres sont employés dans le privé.

In [125]:
print(" The Net salary for everyone are: ")
cat = data.query("prenom == 'Catherine'")
cat
c = cat.assign(revenus_net = lambda x: cat['revenus_bruts'].sub(cat['revenus_bruts']*0.23))

dan = data.query("prenom == 'Daniel'")
dan
d = dan.assign(revenus_net = lambda x: dan['revenus_bruts'].sub(dan['revenus_bruts']*0.23))

mar = data.query("prenom == 'Marie'")
mar
m = mar.assign(revenus_net = lambda x: mar['revenus_bruts'].sub(mar['revenus_bruts']*0.23))

pat = data.query("prenom == 'Patrick'")
pat
p = pat.assign(revenus_net = lambda x: pat['revenus_bruts'].sub(pat['revenus_bruts']*0.15))

frames = [c, d, m, p]
# result = pd.concat(frames, keys=['x', 'y', 'z'])
result = pd.concat(frames)
result

 The Net salary for everyone are: 


,prenom,mois,annee,revenus_bruts,cotisation,revenus_net
0,Catherine,janvier,1979,1600,True,1232.00
1,Catherine,février,1979,1600,True,1232.00
2,Catherine,mars,1979,1600,True,1232.00
3,Catherine,avril,1979,1600,True,1232.00
4,Catherine,mai,1979,1600,True,1232.00
...,...,...,...,...,...,...
1963,Patrick,août,2019,3047,True,2589.95
1964,Patrick,septembre,2019,3047,True,2589.95
1965,Patrick,octobre,2019,3047,True,2589.95
1966,Patrick,novembre,2019,3047,True,2589.95


## 4) c-] Créez une variable contenant la date au format 'MM/YYYY', où MM est un couple de chiffres représentant le mois et YYYY l'année associés au salaire.

In [126]:
import datetime
import dateparser

def dateConverter(month, year):
    return(dateparser.parse( '%s %s' %(month, year), '%b %Y').strftime('%m/%Y'))

dateConverter("février","2010")

'02/2010'

In [127]:
newData = result.assign(fullDate = [*map(dateConverter, result['mois'], result['annee'])])
newData

,prenom,mois,annee,revenus_bruts,cotisation,revenus_net,fullDate
0,Catherine,janvier,1979,1600,True,1232.00,01/1979
1,Catherine,février,1979,1600,True,1232.00,02/1979
2,Catherine,mars,1979,1600,True,1232.00,03/1979
3,Catherine,avril,1979,1600,True,1232.00,04/1979
4,Catherine,mai,1979,1600,True,1232.00,05/1979
...,...,...,...,...,...,...,...
1963,Patrick,août,2019,3047,True,2589.95,08/2019
1964,Patrick,septembre,2019,3047,True,2589.95,09/2019
1965,Patrick,octobre,2019,3047,True,2589.95,10/2019
1966,Patrick,novembre,2019,3047,True,2589.95,11/2019


# 5) Visualisation graphique


## 5) a-] Affichez un nuage de points représentant l'évolution du revenu net des différentes personnes du dataset au fil du temps.

Si possible:

* Affichez seulement les dates de janvier en abcisses, tous les cinq ans
* Ajoutez une légende

In [128]:
print(" Data set for everyone in the month of January between every 5 years: ")
moisAnnee = newData.query("mois == 'janvier' and annee %5 == 0")
moisAnnee

 Data set for everyone in the month of January between every 5 years: 


,prenom,mois,annee,revenus_bruts,cotisation,revenus_net,fullDate
12,Catherine,janvier,1980,1600,True,1232.00,01/1980
72,Catherine,janvier,1985,1616,True,1244.32,01/1985
132,Catherine,janvier,1990,1664,True,1281.28,01/1990
192,Catherine,janvier,1995,1766,True,1359.82,01/1995
252,Catherine,janvier,2000,1802,True,1387.54,01/2000
312,Catherine,janvier,2005,1820,True,1401.40,01/2005
372,Catherine,janvier,2010,1102,False,848.54,01/2010
432,Catherine,janvier,2015,1102,False,848.54,01/2015
504,Daniel,janvier,1980,560,False,431.20,01/1980
564,Daniel,janvier,1985,1854,True,1427.58,01/1985


In [129]:
#gk = g.groupby('prenom') 
moisAnneeGroup = moisAnnee.groupby(['prenom', 'annee']) 
#gk.first()
groupData = moisAnneeGroup.first()
groupData 

mois  revenus_bruts  cotisation  revenus_net fullDate
prenom    annee                                                          
Catherine 1980   janvier           1600        True      1232.00  01/1980
          1985   janvier           1616        True      1244.32  01/1985
          1990   janvier           1664        True      1281.28  01/1990
          1995   janvier           1766        True      1359.82  01/1995
          2000   janvier           1802        True      1387.54  01/2000
          2005   janvier           1820        True      1401.40  01/2005
          2010   janvier           1102       False       848.54  01/2010
          2015   janvier           1102       False       848.54  01/2015
Daniel    1980   janvier            560       False       431.20  01/1980
          1985   janvier           1854        True      1427.58  01/1985
          1990   janvier           1891        True      1456.07  01/1990
          1995   janvier           1949        True      1500.73  01/1995
          2000   janvier           1988        True      1530.76  01/2000
          2005   janvier           2048        True      1576.96  01/2005
          2010   janvier           2110        True      1624.70  01/2010
          2015   janvier           1238       False       953.26  01/2015
Marie     1980   janvier           2244        True      1727.88  01/1980
          1985   janvier           2502        True      1926.54  01/1985
          1990   janvier           2734        True      2105.18  01/1990
          1995   janvier           2901        True      2233.77  01/1995
          2000   janvier           3170        True      2440.90  01/2000
          2005   janvier           3266        True      2514.82  01/2005
          2010   janvier           3399        True      2617.23  01/2010
          2015   janvier           3642        True      2804.34  01/2015
Patrick   1980   janvier           1734        True      1473.90  01/1980
          1985   janvier           1840        True      1564.00  01/1985
          1990   janvier           2051        True      1743.35  01/1990
          1995   janvier           2220        True      1887.00  01/1995
          2000   janvier           2310        True      1963.50  01/2000
          2005   janvier           2575        True      2188.75  01/2005
          2010   janvier           2760        True      2346.00  01/2010
          2015   janvier           2958        True      2514.30  01/2015

In [130]:
print(" Grouped Data set for everyone in the month of January between every 5 years: ")
nom = np.array(groupData.index.levels[0]) # le prenom
nom
#ann = np.array(groupData.index.levels[1])[0:8] # les annee
ann = np.array(groupData.index.levels[1]) # les annee
ann
res = moisAnnee.pivot(index='prenom', columns='annee', values='revenus_net')
res

 Grouped Data set for everyone in the month of January between every 5 years: 


annee,1980,1985,1990,1995,2000,2005,2010,2015
prenom,,,,,,,,
Catherine,1232.00,1244.32,1281.28,1359.82,1387.54,1401.40,848.54,848.54
Daniel,431.20,1427.58,1456.07,1500.73,1530.76,1576.96,1624.70,953.26
Marie,1727.88,1926.54,2105.18,2233.77,2440.90,2514.82,2617.23,2804.34
Patrick,1473.90,1564.00,1743.35,1887.00,1963.50,2188.75,2346.00,2514.30


In [131]:
plt.figure(figsize=(8,8))

indiceAnnee = range(1,len(ann)+1)
#indiceAnnee = range(len(ann))
indiceAnnee
for i in range(len(nom)):
    tab = np.array(res.iloc[i])
    plt.scatter(indiceAnnee, tab)
    plt.plot(indiceAnnee, tab)
plt.legend(res.index)
plt.xticks(indiceAnnee)
plt.ylabel("Net Revenus (en euros)")
plt.xlabel("Annee")
plt.title("Évolution des recettes nettes pour chaque employé dans un délai de 5 ans")
plt.show()

## 5) b-] Affichez un boxplot comparant les distributions de salaires nets des différentes personnes du dataset. Comparez les variations de salaires (écart-type) par personne. Quelle personne a les revenus les plus fluctuants d'après le graphique? Quelle personne a les revenus les plus fluctuants en écart-type? Comment expliquez-vous cette différence?

In [132]:
#dat.groupby(['prenom']).std()['revenus_bruts']
#dat.groupby(['prenom']).mean()['revenus_bruts']
plt.figure(figsize=(8,8))
plt.ylim(200,3000)
sns.boxplot(x = "prenom", y = "revenus_net", data = newData)
plt.show()

In [133]:
print(" The different wage changes (standard deviation) per person are: ")
newData.groupby(['prenom']).std()['revenus_net']

 The different wage changes (standard deviation) per person are: 


prenom
Catherine    249.848153
Daniel       364.092174
Marie        411.761685
Patrick      350.864885
Name: revenus_net, dtype: float64

In [134]:
print(" The person has the most fluctuating income according to the graph is Marie with net revenu for 2300 and above ")

 The person has the most fluctuating income according to the graph is Marie with net revenu for 2300 and above 


In [135]:
print(" The person has the most fluctuating income according to the graph is Marie with net revenu for 411 par year ")

 The person has the most fluctuating income according to the graph is Marie with net revenu for 411 par year 


In [136]:
print("I made the difference by looking at the line (midane line) drawn in the graph")

I made the difference by looking at the line (midane line) drawn in the graph


# 6) Méthodes statistiques

Pour cette question, utilisez les données de la question 4a!


## 6) a-] Modéliser l'évolution du salaire de Patrick au cours du temps à l'aide d'une régression.

In [137]:
# The Plot for Patrick's annual gross income
patAnnee = np.array(patAnnIncome.index) # les annee
patAnnee
plt.figure(figsize=(10,7))
plt.scatter(patAnnee, patAnnIncome)
plt.title('Change in Patricks average annual gross income')
plt.xlabel('Année')
plt.ylabel("Average annual gross income (euros)")
plt.show()

# Calculating the regression line corresponding to Patrick's annual gross income
t = np.array(patAnnIncome).reshape(-1,1)

reg = LinearRegression()
reg.fit(np.array(patAnnee).reshape(-1,1), t)

droite_reg = np.array(reg.coef_*patAnnee+reg.intercept_).reshape(-1,1)

print("Coefficient de la droite de régression", reg.coef_)
print("Constante de régression", reg.intercept_)

# Plotting the regression line in the graph
plt.figure(figsize=(10,7))
plt.scatter(patAnnee, patAnnIncome, c="black", label= 'Patricks average gross income')
plt.plot(patAnnee, droite_reg, c="red", label = 'Regression line')
plt.title('Change in Patricks average annual gross income with Regression line')
plt.xlabel('Année')
plt.ylabel("Average annual gross income (euros)")
plt.legend()
plt.show()

Coefficient de la droite de régression [[34.69651568]]
Constante de régression [-67007.13972125]


## 6) b-] Pouvez-vous prédire ce que gagnera Patrick (en brut) s'il est toujours dans la fonction publique dans 10 ans?

### Answer

### 10 years from now is 2029 (2019 + 10)

In [138]:
pred2029 = reg.coef_*2029+reg.intercept_
print('Patricks average gross income for 10 years from now is : ' + str(pred2029) + ' Euros')

Patricks average gross income for 10 years from now is : [[3392.09059233]] Euros


## 6) c-] Quelle est la valeur de son augmentation annuelle?

### Answer

#### YES we can, see solution below.
#### I already did it before i got your email.  I hope the solution is correct.

#### To find the increase in patricks income for each year from 2029 to 1979,
#### First we find the prediction for each year between 2020 to 2029

In [139]:
pred2020 = reg.coef_*2020+reg.intercept_
pred2021 = reg.coef_*2021+reg.intercept_
pred2022 = reg.coef_*2022+reg.intercept_
pred2023 = reg.coef_*2023+reg.intercept_
pred2024 = reg.coef_*2024+reg.intercept_
pred2025 = reg.coef_*2025+reg.intercept_
pred2026 = reg.coef_*2026+reg.intercept_
pred2027 = reg.coef_*2027+reg.intercept_
pred2028 = reg.coef_*2028+reg.intercept_
pred2029 = reg.coef_*2029+reg.intercept_

mod20 = patAnnIncome.append({'revenus_bruts' : pred2020} , ignore_index=True)
mod21 = mod20.append({'revenus_bruts' : pred2021} , ignore_index=True)
mod22 = mod21.append({'revenus_bruts' : pred2022} , ignore_index=True)
mod23 = mod22.append({'revenus_bruts' : pred2023} , ignore_index=True)
mod24 = mod23.append({'revenus_bruts' : pred2024} , ignore_index=True)
mod25 = mod24.append({'revenus_bruts' : pred2025} , ignore_index=True)
mod26 = mod25.append({'revenus_bruts' : pred2026} , ignore_index=True)
mod27 = mod26.append({'revenus_bruts' : pred2027} , ignore_index=True)
mod28 = mod27.append({'revenus_bruts' : pred2028} , ignore_index=True)
mod29 = mod28.append({'revenus_bruts' : pred2029} , ignore_index=True)
mod29
diffePat = mod29.diff()
diffePat
patMean = diffePat["revenus_bruts"].mean()
patMean
print('Patricks mean increase for each year is ' + str(patMean))

Patricks mean increase for each year is [[33.84181185]]


## 6) d-] Affichez le R² associé à la régression; le modèle vous parait-il correct? Affichez la droite de régression pour vérifier!

In [140]:
total_variance = patAnnIncome['revenus_bruts'].sum()
total_variance
reg.coef_
r_Square = reg.intercept_/total_variance
r_Square

array([-0.69510202])

# 7) Graphe dynamique


Patrick aimerait avoir un aperçu dynamique de ses augmentations cours du temps.

Créez un graphe dynamique qui affiche chacune des moyennes de salaires bruts au fur et à mesure du temps.

Le titre du graphe comportera l'année en cours, et l'augmentation par rapport à l'an passé en pourcentage.

In [141]:
newData['m'] = [str(d)[0:2] for d in newData['fullDate']]

temp_ma = newData.query("prenom == 'Patrick'").groupby(['annee', 'm']).mean()
del temp_ma['cotisation']
del temp_ma['revenus_net']
temp_ma
index_ma = temp_ma.index
#index_ma


In [142]:
df = pd.DataFrame(np.array(temp_ma).reshape(41,12), 
                   columns = temp_ma.index.levels[1]).set_index(temp_ma.index.levels[0])
df

m,01,02,03,04,05,06,07,08,09,10,11,12
annee,,,,,,,,,,,,
1979,1700,1700,1700,1700,1700,1700,1700,1700,1700,1700,1700,1700
1980,1734,1734,1734,1734,1734,1734,1734,1734,1734,1734,1734,1734
1981,1768,1768,1768,1768,1768,1768,1768,1768,1768,1768,1768,1768
1982,1804,1804,1804,1804,1804,1804,1804,1804,1804,1804,1804,1804
1983,1804,1804,1804,1804,1804,1804,1804,1804,1804,1804,1804,1804
1984,1840,1840,1840,1840,1840,1840,1840,1840,1840,1840,1840,1840
1985,1840,1840,1840,1840,1840,1840,1840,1840,1840,1840,1840,1840
1986,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858
1987,1914,1914,1914,1914,1914,1914,1914,1914,1914,1914,1914,1914


In [143]:
mois = ['janvier', 'février', 'mars', 'avril', 'mai', 'juin', 
        'juillet', 'août', 'septembre', 'octobre', 'novembre', 'décembre']

fig = plt.figure(figsize=(15, 10))

#enable interactive graphs
%matplotlib qt

# number of variable
categories=list(df)
N = len(categories)

plt.ion()
plt.figure(figsize=(10,10))
# What will be the angle of each axis in the plot? (we divide the plot / number of variable)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
 
# Initialise the spider plot
ax = plt.subplot(111, polar=True)
 
ax.set_xticklabels(mois)

mois_ord = ['janvier', 'février' , 'mars', 'avril','mai', 'juin','juillet', 
            'août', 'septembre', 'octobre','novembre','décembre']

# Draw one axe per variable + add labels labels yet
plt.xticks(angles, mois_ord , color='grey', size=15)
 
# Draw ylabels
ax.set_rlabel_position(0)
plt.yticks([1000,1500,2000,2500,3000], ["1000","1500","2000","2500","3000"], color="red", size=10)
plt.ylim(1000,3500)

# Plot data
for i in range(41):
    values = df.iloc[i].values
    ax.plot(angles, values, linewidth=1, linestyle='solid')
    fig.canvas.draw()
    fig.canvas.flush_events()
    plt.title(" Le diagramme en radar de l'évolution des revenus bruts de Patrick - Année "+str(patAnnee[i]))
    plt.pause(0.5)

ax.fill(angles, values, 'b', alpha=0)
plt.savefig("TpNoteQuestion7.png")
plt.show()